# Importing the files required

In [1]:
import cv2
import numpy as np

In [2]:
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
from tkinter.simpledialog import askfloat, askinteger
from PIL import Image
from PIL import ImageTk
from functools import partial

# Defining the global variables 

In [3]:
global arr_of_imgs
global arr_new
arr_of_imgs = []

## Convolve and Pad with function definitions

In [4]:
def convolve(arr1, arr2):
    '''Performs the convolution between the two arrays
    arr1 :: any numpy array
    arr2 :: any numpy array
    The sizes of the two arrays must be equal.
    '''
    assert arr1.shape[0]==arr2.shape[0]
    assert arr1.shape[1]==arr2.shape[1] #we must make sure that the shape of the mask and the slice of the image match in size.
    #avg = 0
    #for i in range(0, arr1.shape[0]):
     #   for j in range(0, arr1.shape[1]):
      #      avg += (arr1[i, j] * arr2[i, j])
      # my first code which worked very slow till I discovered that i could flatten the arrays as discussed in class
    return (arr1.flatten()*arr2.flatten()).sum()

In [5]:
#taken from the official documentation of numpy
#https://numpy.org/doc/stable/reference/generated/numpy.pad.html
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 10)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
    

In [6]:
def select():
    global display#for displaying the original image
    global img#image array
    global image 
    path = filedialog.askopenfilename()#asks for path to the image
    if len(path)>0:#if oppath is given then do the following
        img_disp = cv2.imread(path)#read the image as an image array with 3 channels
        img = cv2.cvtColor(img_disp, cv2.COLOR_BGR2RGB)#we conver from RGB to BGR since ImageTk outputs it in BGR format
        image = Image.fromarray(img)#form an object of type Image from the image array img(with channels changed)
        image = ImageTk.PhotoImage(image)#convert to photoimage type for displaying
        if display is None:#if panel has not been assigned anty variable yet then
            display = Label(image = image)
            display.image = image
            display.pack(side = "left", padx = 10, pady = 10)#ddisplaying the image
        else:
            display.configure(image = image)
            display.image = image #diplaying the image
        arr_of_imgs.append(img)#appenign the img to the  image array.

In [7]:
def save():
    #files = [("PNG", "*.png"), ("JPEG", "*.jpg")]

    #filename = filedialog.asksaveasfile(mode='w', defaultextension=files)
    image.save("C:\\Users\\Rohit\\Downloads\\save.jpg")

In [8]:
def Log():
    arr, arr_new = img, img
    for c in range(0, 3):
        scaling =  255 / np.log(1 + np.max(arr[:, :, c]))#we define the scaling factor so that the final values stay in the acceptable range of [0, 255]
        arr_new[:, :, c] = (scaling * np.log(1 + arr[:, :, c])).astype(np.uint8)#saving the final values in the new image array and converting it to type  uint8
    image = Image.fromarray(arr_new)
    image = ImageTk.PhotoImage(image)
    panel_new = Label(image = image)
    panel_new.image = image
    panel_new.pack(side = "right", padx = 10, pady = 10)
    arr_of_imgs.append(arr_new)

In [9]:
def negative():
    arr, arr_new = img, img
    for c in range(0, 3):
        arr_new[:, :, c] = (255  -  arr[:, :, c]).astype(np.uint8)#saving the final values in the new image array and converting it to type  uint8
    image = Image.fromarray(arr_new)
    image = ImageTk.PhotoImage(image)
    panel_new = Label(image = image)
    panel_new.image = image
    panel_new.pack(side = "right", padx = 10, pady = 10)
    arr_of_imgs.append(arr_new)

In [10]:
def Power():
    c = askfloat("Please enter the value of scale", "c = ")
    g = askfloat("Please enter the value of gamma", "gamma = ")
    arr, arr_new = img, img
    for d in range(0, 3):
        scaling = 255/(c*np.max(arr[:, :, d])**g)#we define the scaling factor so that the final values stay in the acceptable range of [0, 255]
        arr_new = (scaling * c * (arr[: , :, d]**g)).astype(np.uint8)# performing the operation on the image array
    image = Image.fromarray(arr_new)
    image = ImageTk.PhotoImage(image)
    panel_new = Label(image = image)
    panel_new.image = image
    panel_new.pack(side = "right", padx = 10, pady = 10)
    arr_of_imgs.append(arr_new)  

In [11]:
def Blur():
    kernel_size = askinteger("Please enter the value of kernel_size", "kernel_size = ")
    r = kernel_size//2 #integer ddivsionto get the width of the padding
    arr, arr_new = img, img
    mask = np.ones((kernel_size, kernel_size))/sum(sum(np.ones((kernel_size, kernel_size))))#defining the kernel and is normalised so that final values don't exceed 255
    #mask = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    for c in range(0, 3):
        #we now perform the convolution for the image
        #we first pad the image array with zeros of length/width r and then move the mask across the image
        #we use the previously defined convolve function to find the convoltion between the slice of the image array and the mask.
        pad_arr = np.pad(arr[:, :, c], r, pad_with, padder =0)#stores the podded array
        for i in range(0, pad_arr.shape[0] - kernel_size+1):
            for j in range(0, pad_arr.shape[1] - kernel_size+1):
                arr2conv = pad_arr[i:i+kernel_size, j:j+kernel_size]#choosing the slice of the image array
                arr_new[i, j, c] = convolve(arr2conv, mask)#calling the convolve function to convolve the 2
    image = Image.fromarray(arr_new.astype(np.uint8))
    image = ImageTk.PhotoImage(image)
    panel_new = Label(image = image)
    panel_new.image = image
    panel_new.pack(side = "right", padx = 10, pady = 10)
    arr_of_imgs.append(arr_new)

In [12]:
def sharpen():
    kernel_size = askinteger("Please enter the value of kernel_size", "kernel_size = ")
    r = kernel_size//2 #integer ddivsionto get the width of the padding
    arr, arr_new = img, img
    mask = -np.ones((kernel_size, kernel_size))
    mask[r, r] = kernel_size**2 - 1#normal laplacian has the centre element such that the net sum is 0.
    #mask = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    for c in range(0, 3):
        #we now perform the convolution for the image
        #we first pad the image array with zeros of length/width r and then move the mask across the image
        #we use the previously defined convolve function to find the convoltion between the slice of the image array and the mask.
        pad_arr = np.pad(arr[:, :, c], r, pad_with, padder =0)
        for i in range(0, pad_arr.shape[0] - kernel_size+1):
            for j in range(0, pad_arr.shape[1] - kernel_size+1):
                arr2conv = pad_arr[i:i+kernel_size, j:j+kernel_size]#choosing the slice of the image array
                arr_new[i, j, c] = convolve(arr2conv, mask)#calling the convolve function to convolve the 2
        #arr_new[:, :, c] = arr_new[: ,:, c] - np.min(arr_new[:, :, c])
        #arr_new[:, :, c] = arr_new[:, :, c]*(255/np.max(arr_new[:, :, c]))
    image = Image.fromarray(arr_new.astype(np.uint8))
    image = ImageTk.PhotoImage(image)
    panel_new = Label(image = image)
    panel_new.image = image
    panel_new.pack(side = "right", padx = 10, pady = 10)
    arr_of_imgs.append(arr_new)

In [13]:
def Eq():
    arr_og, arr_new = img, img
    for c in range(0, 3):
        #took help from this site https://towardsdatascience.com/histogram-equalization-a-simple-way-to-improve-the-contrast-of-your-image-bcd66596d815
        arr = arr_og[:, :, c]
        hist, _ = np.histogram(arr.flatten(), 256, [0, 256]) #defining the histogram for the given array
        cdf = np.cumsum(hist)#defining the cummulative function or the cdf of the pixels
        arr_new_2d = (cdf - cdf.min())*255/(cdf.max()-cdf.min()) #finding the difference and normalising it so the maximum value is till only 255
        arr_new[:, : , c] = arr_new_2d[arr]#fetching the final image array after equilisation
    arr_of_imgs.append(arr_new)
    image = Image.fromarray(arr_new.astype(np.uint8))
    image = ImageTk.PhotoImage(image)
    panel_new = Label(image = image)
    panel_new.image = image
    panel_new.pack(side = "right", padx = 10, pady = 10)


In [14]:
def undo_all():
    arr_new = img
    
    image = Image.fromarray(arr_new)
    image = ImageTk.PhotoImage(image)
    panel_new = Label(image = image)
    panel_new.image = image
    panel_new.pack(side = "right", padx = 10, pady = 10)

In [18]:
root = Tk()
display = None#initialise the varible to none
#below we define all the buttons required for the gui
#btn = Button(root, text  ="Save my image!!!!!(Gives error :( )", command = save)
#btn.pack(side = "bottom", fill = "both", expand="yes", padx = 35, pady = 16)
btn = Button(root, text  ="Log transform", command = Log)
btn.pack(side = "bottom", fill = "both", expand="yes", padx = 11, pady = 11)
btn = Button(root, text  ="Power transform", command = Power)
btn.pack(side = "bottom", fill = "both", expand="yes", padx = 11, pady = 12)
btn = Button(root, text  ="Blur", command = Blur)
btn.pack(side = "bottom", fill = "both", expand="yes", padx = 30, pady = 13)
btn = Button(root, text  ="Equilize", command = Eq)
btn.pack(side = "bottom", fill = "both", expand="yes", padx = 30, pady = 14)
btn = Button(root, text  ="Sharpen", command = sharpen)
btn.pack(side = "bottom", fill = "both", expand="yes", padx = 35, pady = 15)
btn = Button(root, text  ="Negative of my image", command = negative)
btn.pack(side = "bottom", fill = "both", expand="yes", padx = 35, pady = 10)
btn = Button(root, text  ="Select image:: Please select an image first!!", command = select)
btn.pack(side = "bottom", fill = "both", expand="yes", padx = 10, pady = 17)
root.mainloop()
arr_of_imgs = []

# Generating the requirements files

In [16]:
import session_info

In [17]:
session_info.show()

------------------------------------------------------------------------------------------------------------------------